# Environment Setup

In [9]:
!git clone https://github.com/anantgupta129/TorcHood.git
%cd TorcHood  

Cloning into 'TorcHood'...
remote: Enumerating objects: 648, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 648 (delta 40), reused 43 (delta 23), pack-reused 587
Receiving objects: 100% (648/648), 148.48 KiB | 231.00 KiB/s, done.
Resolving deltas: 100% (422/422), done.
/home/watcherai/.dev/era/TorcHood


In [10]:
from IPython.display import clear_output

!python setup.py sdist
!pip install .

clear_output()
%cd ..

/home/watcherai/.dev/era


# Importing Libaries

In [11]:
import warnings
warnings.filterwarnings("ignore")

import os 
import torch

import lightning.pytorch as pl
from torchood.configs.bilang_config import get_config
from torchood.data.opus_datamodule import OpusBooksDataModule
from torchood.models.bilang_module import BiLangLitModule

In [12]:
config = get_config()
config["batch_size"] = 32

In [13]:
datamodule = OpusBooksDataModule(config, 8, True)

In [14]:
datamodule.prepare_data()
datamodule.setup("")

Tokenizer found. Loading from file...
Tokenizer found. Loading from file...
[x] Maximum Length of Sentence:-
	[*] source sentence: 309
	[*] target sentence: 274


In [15]:
model = BiLangLitModule(config["lr"], config, datamodule.tokenizer_src, datamodule.tokenizer_tgt)

In [16]:
from lightning.pytorch.callbacks import (
    LearningRateMonitor,
    RichProgressBar,
)
from lightning.pytorch.loggers import WandbLogger


callbacks = [
    RichProgressBar(leave=True),
    LearningRateMonitor(logging_interval="step"),
]
logger = WandbLogger(project="Transformers-BiLang")
trainer = pl.Trainer(callbacks=callbacks, 
                    logger=logger, 
                    max_epochs=10, 
                    accelerator = "gpu")
trainer.fit(model=model, datamodule=datamodule)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: anantgupta129. Use `wandb login --relogin` to force relogin


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name            ┃ Type             ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ net             │ Transformer      │ 75.1 M │
│ 1 │ criterion       │ CrossEntropyLoss │      0 │
│ 2 │ char_error_rate │ CharErrorRate    │      0 │
│ 3 │ word_error_rate │ WordErrorRate    │      0 │
│ 4 │ bleu_score      │ BLEUScore        │      0 │
└───┴─────────────────┴──────────────────┴────────┘

Trainable params: 75.1 M                                                                                           
Non-trainable params: 0                                                                                            
Total params: 75.1 M                                                                                               
Total estimated model params size (MB): 300

Output()

...

Output()

Output()

Output()

wandb: Network error (ReadTimeout), entering retry loop.


`Trainer.fit` stopped: `max_epochs=10` reached.
